<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №3</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

## Доверительный интервал уровня $1 - \alpha$
### Теория

Есть две выборки $X_1, X_2 \dots X_n \sim N(\mu_1, \sigma_1^2)$ и $Y_1, Y_2 \dots Y_m \sim N(\mu_2, \sigma_2^2)$. Оцениваем $\theta = \cfrac{\sigma_1^2}{\sigma_2^2}$, при этом $\mu_1$ и $\mu_2$ нам известны. 

Первым делом введём слуайную величину $U_1$

In [1]:
print("cringe")

cringe
